In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import pickle
import msgpack
import time
import pandas as pd
from gensim.models import Word2Vec, FastText

from train_dsm import GeneratorSentences
from train_dsm import GeneratorSentencesCompounds
from train_dsm import train_dsm

In [4]:
import os
import sys
from train_dsm import train_dsm
from multiprocessing import Process
from gensim.models import FastText

logs_path = './workdir/logs/subprocesses/'


def init_subprocess_logging():
    import sys
    sys.stdout = open(os.path.join(logs_path, str(os.getpid()) + ".stdout"), "a")
    sys.stderr = open(os.path.join(logs_path, str(os.getpid()) + ".stderr"), "a")
    
    
def run_custom(func, *args, **kwargs):
    init_subprocess_logging()
    return func(*args, **kwargs)


def get_logs(proc, tp='.stderr', tail=100):
    proc_logs_path = os.path.join(logs_path, str(proc.pid) + tp)
    with open(proc_logs_path, 'rb') as f:
        try:
            f.seek(-tail, os.SEEK_END)
        except OSError:
            pass
        finally:
            log = f.read()
    
    return log.decode('utf-8')

# Train model without compounds

In [ ]:
import gensim

#parse_path = '/notebook/projects/compounds/workdir/parse1/'
parse_path = '/notebook/projects/compounds/workdir/parse5/'
model = gensim.models.Word2Vec(GeneratorSentences(parse_path), min_count=5, size=100, workers=10)
model.save('./model_word2vec_no_compounds')

In [41]:
model.wv.most_similar('мыть')

[('чистить', 0.9070878624916077),
 ('помыть', 0.8470351696014404),
 ('мочить', 0.8346104621887207),
 ('протирать', 0.8309078216552734),
 ('намазывать', 0.8274841904640198),
 ('мыться', 0.8230193257331848),
 ('полоскать', 0.8215987682342529),
 ('вытирать', 0.8136080503463745),
 ('мазать', 0.8098058700561523),
 ('замачивать', 0.8080874681472778)]

# Train word2vec

In [6]:
import gensim

parse_path = '/notebook/projects/compounds/workdir/parse5/'
model = gensim.models.Word2Vec(GeneratorSentencesCompounds(parse_path), min_count=5, size=500, workers=5)
model.save('./workdir/models/model_word2vec_compounds5')

In [7]:
model.wv.most_similar('автор_программа')

/root/.pyenv/versions/3.6.7/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('программа', 0.5038685202598572),
 ('телепрограмма', 0.4862660765647888),
 ('телепередача', 0.4710542559623718),
 ('соведущий', 0.4702419340610504),
 ('телеигра', 0.4528096616268158),
 ('колумнист', 0.42590969800949097),
 ('рубрика', 0.4214072525501251),
 ('идеолог', 0.4181630313396454),
 ('модератор', 0.4072508215904236),
 ('телевикторина', 0.4044981896877289)]

In [ ]:
parse_path = '/notebook/projects/compounds/workdir/parse5'
full_data = list(GeneratorSentencesCompounds(parse_path))

In [5]:
compounds_path = './workdir/annotation_katya_ref_v2.csv'
data_path = './workdir/parse5'
save_path = './workdir/models/model_fast2vec_test'
model = Word2Vec(size=300, min_count=2, workers=8)

train_dsm(compounds_path=compounds_path, 
          data_path=data_path, 
          save_path=save_path, 
          model=model, 
          epochs=5,
          strategy=1)

In [6]:
model.save('./workdir/models/model_word2vec_test1')

# Train FastText

In [5]:
compounds_path = './workdir/annotation_katya_ref_v2.csv'
data_path = './workdir/parse5'
save_path = './workdir/models/model_fasttext_300_6'
model = FastText(size=300, min_count=2, workers=8)
proc_fasttext_1 = Process(target=run_custom, 
               args=[train_dsm],
               kwargs=dict(compounds_path=compounds_path, 
                                    data_path=data_path, 
                                    save_path=save_path, 
                                    model=model, 
                                    epochs=5))
proc_fasttext_1.start()